# Computational Theory 

## Problem 1 — Binary Words and Operations

Implement the SHA-256 bitwise functions: `Parity`, `Ch`, `Maj`, `Sigma0`, `Sigma1`, `sigma0`, `sigma1`.

In this problem, I implemented the core logical and bitwise functions that make up the foundation of the SHA-256 algorithm

Each function performs specific bit manipulations like rotations and XOR operations to ensure strong diffusion and unpredictability, which are key properties in cryptographic hash design.  

In this section, I implemented the six main logical and bitwise functions defined on page 10 sections 4.1.1 and 4.1.2 of the Secure Hash Standard. This is the core document for this notebook and assesment a link to it is included in the README.

The implemented functions are:
- **Ch(x, y, z)** – The “choose” function, used to select bits based on the value of *x*  
- **Maj(x, y, z)** – The “majority” function, outputs the majority bit among *x*, *y*, *z*  
- **Σ₀(x)** and **Σ₁(x)** – Perform rotations and XOR operations to achieve bit dispersion  
- **σ₀(x)** and **σ₁(x)** – Smaller versions of Σ, also involving shifts and rotations  
- **Parity(x, y, z)** – Used in SHA-1, but included here for comparison and testing  

Each function operates on **32-bit words**, using NumPy’s `uint32` type to ensure the same behavior as the specification. The formulas come directly from the standard:

\[
\begin{align*}
Ch(x, y, z) &= (x \land y) \oplus (\lnot x \land z) \\
Maj(x, y, z) &= (x \land y) \oplus (x \land z) \oplus (y \land z) \\
Σ_0(x) &= ROTR^2(x) \oplus ROTR^{13}(x) \oplus ROTR^{22}(x) \\
Σ_1(x) &= ROTR^6(x) \oplus ROTR^{11}(x) \oplus ROTR^{25}(x) \\
σ_0(x) &= ROTR^7(x) \oplus ROTR^{18}(x) \oplus SHR^3(x) \\
σ_1(x) &= ROTR^{17}(x) \oplus ROTR^{19}(x) \oplus SHR^{10}(x)
\end{align*}
\]



Below we provide:

- Clear docstrings for each function.
- Explanations of the logical behaviour (algorithmic steps)
- Tests comparing small examples and showing values in hex.


### Background Reading

While researching how cryptographic hash functions work, I came across an article called [*Cryptography Hash Method MD2 (Message Digest 2) – Step-by-Step Explanation Made Easy with Python*](https://nickthecrypt.medium.com/cryptography-hash-method-md2-message-digest-2-step-by-step-explanation-made-easy-with-python-10faa2e35e85) by Nick The Crypt. Although that article focuses on the older MD2 algorithm, it helped me understand the overall logic behind hash functions how they mix and transform bits to make outputs unpredictable. MD2 and SHA-256 are quite different in their design (MD2 uses substitution tables, while SHA-256 relies on logical operations and rotations), but the core idea of achieving diffusion and non-reversibility is the same. Reading it gave me a better appreciation for why SHA-256 uses helper functions like `Ch`, `Maj`, and the Σ/σ functions I implemented here.


### Logical Explanation – Bitwise Operations and Helper Functions

The functions below form the mathematical backbone of cryptographic hash algorithms like SHA-256.  
They rely on **bitwise operations** that manipulate the binary representation of 32-bit integers to create diffusion (mixing of input bits).

1. **_rotr(x, n):**  
   This function performs a *bitwise rotation* of a 32-bit integer `x` to the right by `n` positions.  
   Unlike a normal shift, the bits that “fall off” on the right side are wrapped around to the left side.  
   This operation preserves all bits while changing their order an important property for reversible bit mixing.

2. **Parity(x, y, z):**  
   This computes the bitwise XOR (exclusive OR) of three values.  
   Each bit in the result is 1 if an odd number of corresponding bits among `x`, `y`, and `z` are 1.  
   It’s a simple way to measure “odd parity” among bits and is used to combine data unpredictably.

3. **Ch(x, y, z):**  
   Also called the *choose* function.  
   It picks bits from `y` or `z` depending on whether the corresponding bit in `x` is 1 or 0.  
   This is a conditional operation at the bit level similar to `if (x) choose y else choose z`.

4. **Maj(x, y, z):**  
   Also known as the *majority* function.  
   For each bit position, it outputs 1 if at least two of the three inputs have that bit set.  
   This simulates a logical majority vote for every bit position.

In [1]:

import numpy as np # Import NumPy for 32-bit unsigned integer support

# Helper for rotating 32-bit integers
def _rotr(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right (32-bit) helper."""
     # Shift bits right by n and wrap shifted bits from the right end to the left
    return np.uint32((x >> n) | (x << (32 - n)))

# Bitwise operations used in SHA-like algorithms
def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Parity(x,y,z) = x XOR y XOR z."""
     # XOR returns 1 only when an odd number of inputs have 1 in that bit position
    return np.uint32(x ^ y ^ z)

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z)."""
    # For each bit in x: if bit is 1 → choose y; if bit is 0 → choose z
    return np.uint32((x & y) ^ (~x & z))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z)."""
     # Returns 1 if the majority (two or more) of input bits are 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


### Logical Explanation – Sigma and sigma Functions

The Sigma functions introduce **nonlinearity** and **diffusion** into the hashing process.  
They combine **bit rotations** and **bit shifts** to ensure that small changes in the input produce large, unpredictable changes in the output a property known as the *avalanche effect*.

- **Σ0(x)** and **Σ1(x)**:  
  These are used in the *main compression function* of SHA algorithms.  
  Each function applies multiple rotate-right operations and XORs the results together.  
  This mixes bits from distant positions within the 32-bit word, increasing complexity.

- **σ0(x)** and **σ1(x):**  
  These are the *small sigma* functions, used when expanding the message schedule in SHA algorithms.  
  They use a combination of rotate-right and right-shift operations (`>>`).  
  The right shift (`SHR`) introduces zeros from the left side, losing information which helps achieve more randomness and diffusion during message expansion.


In [2]:
# Uppercase Sigma functions (used in main compression)
def Sigma0(x: np.uint32) -> np.uint32:
    """Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)."""
      # Combine rotated versions of x to achieve bit diffusion
    return np.uint32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)."""
    # Similar to Sigma0 but uses different rotation constants
    return np.uint32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

# Lowercase sigma functions (used for message schedule)
def sigma0(x: np.uint32) -> np.uint32:
    """σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)."""
     # Use rotate-right and logical right shift to mix bits in the message schedule
    return np.uint32(_rotr(x, 7) ^ _rotr(x, 18) ^ (x >> 3))

def sigma1(x: np.uint32) -> np.uint32:
    """σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)."""
      # Another mix function for message expansion with different bit positions
    return np.uint32(_rotr(x, 17) ^ _rotr(x, 19) ^ (x >> 10))


### Testing and Verification

The cell below verifies that all previously defined functions behave as expected.  
By using fixed 32-bit hexadecimal test values for `x`, `y`, and `z`, we can visually confirm the correctness of each operation.

Each `print()` statement displays the function output in **hexadecimal format** for easy reading.  
If the outputs are consistent with the expected bitwise behaviors (e.g., XOR, AND, rotations), then the implementation is correct.

These tests act as a **sanity check** before integrating the functions into larger cryptographic processes.


In [3]:
# Quick test to make sure everything works
x = np.uint32(0x12345678)
y = np.uint32(0x9abcdef0)
z = np.uint32(0x0f1e2d3c)

# Print results of each function in hexadecimal format for clarity
print("Parity:  ", f"{Parity(x,y,z):#010x}")
print("Ch:      ", f"{Ch(x,y,z):#010x}")
print("Maj:     ", f"{Maj(x,y,z):#010x}")
print("Sigma0:  ", f"{Sigma0(x):#010x}")
print("Sigma1:  ", f"{Sigma1(x):#010x}")
print("sigma0:  ", f"{sigma0(x):#010x}")
print("sigma1:  ", f"{sigma1(x):#010x}")


Parity:   0x8796a5b4
Ch:       0x1f3e7f74
Maj:      0x1a3c5e78
Sigma0:   0x66146474
Sigma1:   0x3561abda
sigma0:   0xe7fce6ee
sigma1:   0xa1f78649


### Test Results and Explanation 

As you can see, the above results appear correct and consistent with what we’d expect from these logical and bitwise operations. Each function gives a distinct output, which shows that the rotations, shifts, and XOR/AND combinations are behaving properly.

Parity(x, y, z) gives the XOR of all three values, evenly mixing their bits.

Ch(x, y, z) acts like a conditional function, choosing bits from y or z based on the value of x.

Maj(x, y, z) produces the majority bit from the three inputs.

Sigma0 and Sigma1 apply multiple right rotations and XORs to increase bit diffusion, which is key for the non-linearity in SHA-256.

sigma0 and sigma1 use a mix of rotations and right shifts to help expand the message schedule.

## Problem 2 — Fractional Parts of Cube Roots

In this problem, we’ll calculate the SHA-256 round constants, which are derived from the fractional parts of the cube roots of the first 64 prime numbers.

These constants appear at the bottom of page 11 of the Secure Hash Standard which we mentioned in problem 1.

We’ll compute them using numpy step by step, then compare our results to the official values from the secure hash standard.

### Generate first 64 primes

In [5]:
# Function to generate the first n prime numbers 
# n in this case will be the first 64 primes 
import numpy as np

def primes(n):
    primes_list = []
    num = 2
    while len(primes_list) < n:
        for p in primes_list:
            if num % p == 0:
                break
        else:
            primes_list.append(num)
        num += 1
    return np.array(primes_list)

# Generate the first 64 primes
prime_nums = primes(64)
prime_nums


array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311])

As we can see the function above outputs the first 64 primes.

### Cube roots

Now, for each prime number, we calculate its cube root. The cube root is the number which, multiplied by itself three times, equals the original number see cell below

In [6]:
cube_roots = np.cbrt(prime_nums)  # cube root of each prime
cube_roots[:64]  # preview the first 64 cube roots

array([1.25992105, 1.44224957, 1.70997595, 1.91293118, 2.22398009,
       2.35133469, 2.57128159, 2.66840165, 2.84386698, 3.07231683,
       3.14138065, 3.33222185, 3.44821724, 3.50339806, 3.60882608,
       3.75628575, 3.89299642, 3.93649718, 4.0615481 , 4.14081775,
       4.1793392 , 4.29084043, 4.36207067, 4.4647451 , 4.59470089,
       4.65700951, 4.68754815, 4.7474594 , 4.77685618, 4.83458813,
       5.0265257 , 5.07875308, 5.15513674, 5.18010147, 5.30145919,
       5.32507402, 5.39469071, 5.46255557, 5.50687845, 5.57205466,
       5.63574079, 5.65665283, 5.75896522, 5.77899657, 5.81864787,
       5.83827246, 5.95334181, 6.06412699, 6.1001702 , 6.11803317,
       6.15344949, 6.20582179, 6.22308425, 6.30799355, 6.35786118,
       6.40695858, 6.45531481, 6.47127363, 6.51868392, 6.54991162,
       6.56541443, 6.6418522 , 6.74599671, 6.77516895])



Above, we calculated the **cube roots** of the first 64 prime numbers using NumPy’s built-in function `np.cbrt()`.  
This gives us the decimal (floating-point) representation of each cube root.

The concept is the **cube root** of a number is the value that, when multiplied by itself three times, gives the original number back.

The implemented operation is:

- **Cube Root (³√x)** – Finds the value such that **(³√x)³ = x**

Here are a few examples:

- **³√2 ≈ 1.2599**  
- **³√3 ≈ 1.4422**  
- **³√5 ≈ 1.7099**  
- **³√7 ≈ 1.9129**

These results above show the cube roots of the first 64 prime numbers.  
For the next step, we’ll extract only the **fractional part** (the digits after the decimal point), since that’s what’s used to generate the constants in the **Secure Hash Standard (SHA-256)**.




### Fractional parts of cube roots


In [7]:
fractional_parts = cube_roots % 1  # keep only the part after the decimal
fractional_parts[:64]  # preview first 64 fractional parts

array([0.25992105, 0.44224957, 0.70997595, 0.91293118, 0.22398009,
       0.35133469, 0.57128159, 0.66840165, 0.84386698, 0.07231683,
       0.14138065, 0.33222185, 0.44821724, 0.50339806, 0.60882608,
       0.75628575, 0.89299642, 0.93649718, 0.0615481 , 0.14081775,
       0.1793392 , 0.29084043, 0.36207067, 0.4647451 , 0.59470089,
       0.65700951, 0.68754815, 0.7474594 , 0.77685618, 0.83458813,
       0.0265257 , 0.07875308, 0.15513674, 0.18010147, 0.30145919,
       0.32507402, 0.39469071, 0.46255557, 0.50687845, 0.57205466,
       0.63574079, 0.65665283, 0.75896522, 0.77899657, 0.81864787,
       0.83827246, 0.95334181, 0.06412699, 0.1001702 , 0.11803317,
       0.15344949, 0.20582179, 0.22308425, 0.30799355, 0.35786118,
       0.40695858, 0.45531481, 0.47127363, 0.51868392, 0.54991162,
       0.56541443, 0.6418522 , 0.74599671, 0.77516895])

Above we remove the whole number part and keep only the decimals. For example, if a cube root is 5.874, we only keep 0.874. This is the part that will be turned into constants. 

### Conversion to match constants

Now that we have the **fractional part** of each cube root, we need to turn it into the **constants used in SHA-256**.  

Here’s what we are doing in simple terms:

1. **Multiply by 2³²**  
   - The fractional part is a number between 0 and 1 (e.g., 0.259921).  
   - Multiplying by 2³² scales it to a **32-bit integer range**, because SHA-256 constants are **32-bit words**.

2. **Convert to 32-bit unsigned integers**  
   - This truncates any extra decimal part and ensures the value fits exactly into 32 bits.  
   - We use `np.uint32` to match the format required by the SHA-256 standard.

3. **Convert to hexadecimal format**  
   - SHA-256 constants are written in hexadecimal (base 16).  
   - Using `hex()` turns our integers into hex strings, which we can then compare to the **official constants** from the Secure Hash Standard.

**Example:**  
If the fractional part is `0.259921`, then we scale it to a 32-bit integer by multiplying by 2^32:

0.259921 × 2^32 ≈ 1116352408

Converting this integer to hexadecimal gives:

1116352408 → 0x428a2f98

This matches the first constant in the SHA-256 standard.  

By repeating this for all 64 primes, we generate the **full set of SHA-256 constants**.

In [16]:
# Multiply by 2^32 and convert to 32-bit unsigned integers
constants = (fractional_parts * (2**32)).astype(np.uint32)
hex_constants = [hex(c) for c in constants]
hex_constants[:64]  # preview first 64 constants

['0x428a2f98',
 '0x71374491',
 '0xb5c0fbcf',
 '0xe9b5dba5',
 '0x3956c25b',
 '0x59f111f1',
 '0x923f82a4',
 '0xab1c5ed5',
 '0xd807aa98',
 '0x12835b01',
 '0x243185be',
 '0x550c7dc3',
 '0x72be5d74',
 '0x80deb1fe',
 '0x9bdc06a7',
 '0xc19bf174',
 '0xe49b69c1',
 '0xefbe4786',
 '0xfc19dc6',
 '0x240ca1cc',
 '0x2de92c6f',
 '0x4a7484aa',
 '0x5cb0a9dc',
 '0x76f988da',
 '0x983e5152',
 '0xa831c66d',
 '0xb00327c8',
 '0xbf597fc7',
 '0xc6e00bf3',
 '0xd5a79147',
 '0x6ca6351',
 '0x14292967',
 '0x27b70a85',
 '0x2e1b2138',
 '0x4d2c6dfc',
 '0x53380d13',
 '0x650a7354',
 '0x766a0abb',
 '0x81c2c92e',
 '0x92722c85',
 '0xa2bfe8a1',
 '0xa81a664b',
 '0xc24b8b70',
 '0xc76c51a3',
 '0xd192e819',
 '0xd6990624',
 '0xf40e3585',
 '0x106aa070',
 '0x19a4c116',
 '0x1e376c08',
 '0x2748774c',
 '0x34b0bcb5',
 '0x391c0cb3',
 '0x4ed8aa4a',
 '0x5b9cca4f',
 '0x682e6ff3',
 '0x748f82ee',
 '0x78a5636f',
 '0x84c87814',
 '0x8cc70208',
 '0x90befffa',
 '0xa4506ceb',
 '0xbef9a3f7',
 '0xc67178f2']

### Official constants 

Below we see the official SHA-256 constants. We check to see how many of our converted 64 primes match and print the result.

In [17]:
official_constants_hex = [
    "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
    "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
    "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
    "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
    "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
    "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
    "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
    "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"
]

matches = [our == off for our, off in zip(hex_constants, official_constants_hex)]
sum(matches), len(matches)  # how many match out of 64


(0, 64)